In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_log_error
import lightgbm as lgb
from bayes_opt import BayesianOptimization
pd.set_option('mode.chained_assignment',  None)
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거


In [29]:
train_data = pd.read_csv('final_train_data.csv')
test_data = pd.read_csv('final_test_data.csv')

In [30]:
train_data.head()

,date,store_nbr,family,sales,onpromotion,dcoilwtico,transactions,type_y,cluster,Year,Month,Weekday,trend,NewYearsDay,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre,no_work_day
0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
1,2013-01-01,1,BABY CARE,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
2,2013-01-01,1,BEAUTY,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
3,2013-01-01,1,BEVERAGES,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
4,2013-01-01,1,BOOKS,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True


In [31]:
test_data.head()

,date,store_nbr,family,onpromotion,dcoilwtico,transactions,type_y,cluster,Year,Month,Weekday,trend,NewYearsDay,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre,no_work_day
0,2017-08-16,1,AUTOMOTIVE,0,46.8,1448.259579,D,13,2017,8,2,1688,False,0,0,0,0,0,0
1,2017-08-16,1,BABY CARE,0,46.8,1448.259579,D,13,2017,8,2,1688,False,0,0,0,0,0,0
2,2017-08-16,1,BEAUTY,2,46.8,1448.259579,D,13,2017,8,2,1688,False,0,0,0,0,0,0
3,2017-08-16,1,BEVERAGES,20,46.8,1448.259579,D,13,2017,8,2,1688,False,0,0,0,0,0,0
4,2017-08-16,1,BOOKS,0,46.8,1448.259579,D,13,2017,8,2,1688,False,0,0,0,0,0,0


In [32]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 20 columns):
 #   Column           Dtype  
---  ------           -----  
 0   date             object 
 1   store_nbr        int64  
 2   family           object 
 3   sales            float64
 4   onpromotion      int64  
 5   dcoilwtico       float64
 6   transactions     float64
 7   type_y           object 
 8   cluster          int64  
 9   Year             int64  
 10  Month            int64  
 11  Weekday          int64  
 12  trend            int64  
 13  NewYearsDay      bool   
 14  Earthquake       int64  
 15  Cyber Monday     int64  
 16  Football         int64  
 17  Black Friday     int64  
 18  Dia de la Madre  int64  
 19  no_work_day      bool   
dtypes: bool(2), float64(3), int64(12), object(3)
memory usage: 425.3+ MB


In [33]:
# 데이터 형 변환
train_data = train_data.astype({'store_nbr':'object', 'Earthquake':'bool', 'Cyber Monday':'bool', 'Football':'bool', 'Black Friday':'bool','Dia de la Madre':'bool'})
test_data = test_data.astype({'store_nbr':'object', 'Earthquake':'bool', 'Cyber Monday':'bool', 'Football':'bool', 'Black Friday':'bool','Dia de la Madre':'bool'})

**train, vaild, test data split**

In [34]:
# data split
sub_train_data = train_data[train_data.date <= '2016-07-31']
valid_data = train_data[train_data.date >= '2016-08-01']

In [35]:
print(train_data.shape)
print(sub_train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(3054348, 20)
(2368278, 20)
(686070, 20)
(28512, 19)


In [36]:
# 검증 과정에서 사용할 데이터
sub_train_data.drop(columns = ['date'], inplace = True)
valid_data.drop(columns = ['date'], inplace = True)

# 최종 모델 훈련에서 사용할 데이터
train_data.drop(columns = ['date'], inplace = True)
test_data.drop(columns = ['date'], inplace = True)

In [37]:
X_sub_train_data = sub_train_data.drop(columns = ['sales'])
y_sub_train_data = sub_train_data['sales']

X_valid_data = valid_data.drop(columns = ['sales'])
y_valid_data = valid_data['sales']

In [38]:
X_train_data = train_data.drop(columns = ['sales'])
y_train_data = train_data['sales']

**데이터 전처리**


In [39]:
# OneHotEncoding(get_dummies)
X_sub_train_data_trans = pd.get_dummies(X_sub_train_data, drop_first = True)
X_valid_data_trans = pd.get_dummies(X_valid_data, drop_first = True)
X_train_data_trans = pd.get_dummies(X_train_data, drop_first = True)
test_data_trans = pd.get_dummies(test_data, drop_first = True)

In [40]:
# 검증과정에서 사용할 데이터
# MinMax
sub_train_min = np.min(X_sub_train_data[['transactions', 'onpromotion', 'dcoilwtico']])
sub_train_max = np.max(X_sub_train_data[['transactions', 'onpromotion', 'dcoilwtico']])
sub_train_range = sub_train_max - sub_train_min

# 변환
X_sub_train_data_trans[['transactions', 'onpromotion', 'dcoilwtico']] = (X_sub_train_data[['transactions', 'onpromotion', 'dcoilwtico']] - sub_train_min) / sub_train_range
X_valid_data_trans[['transactions', 'onpromotion', 'dcoilwtico']] = ((X_valid_data[['transactions', 'onpromotion', 'dcoilwtico']] - sub_train_min) / sub_train_range)

# 최종 훈련에 사용할 데이터
train_min = np.min(X_train_data[['transactions', 'onpromotion', 'dcoilwtico']])
train_max = np.max(X_train_data[['transactions', 'onpromotion', 'dcoilwtico']])
train_range = train_max - train_min

# 변환
X_train_data_trans[['transactions', 'onpromotion', 'dcoilwtico']] = (X_train_data[['transactions', 'onpromotion', 'dcoilwtico']] - train_min) / train_range
test_data_trans[['transactions', 'onpromotion', 'dcoilwtico']] = ((test_data[['transactions', 'onpromotion', 'dcoilwtico']] - train_min) / train_range)


**Linear Regression**

In [41]:
# modeling
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_sub_train_data_trans, y_sub_train_data)

LinearRegression()

In [42]:
# predict
train_pred = model.predict(X_sub_train_data_trans)
valid_pred = model.predict(X_valid_data_trans)

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0

In [43]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

훈련데이터 rmsle:  2.4489112606797234
검증데이터 rmsle:  2.3055302057203173


**light GBM**

- error
    - Do not support special JSON characters in feature name.

In [16]:
import re
X_sub_train_data_trans = X_sub_train_data_trans.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_valid_data_trans = X_valid_data_trans.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train_data_trans = X_train_data_trans.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_data_trans = test_data_trans.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [18]:
# install
#!pip install lightgbm

- reference
    - https://greatjoy.tistory.com/72
    - https://smecsm.tistory.com/133
    - http://machinelearningkorea.com/2019/09/29/lightgbm-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0/
    
    
    
- 주요 파라미터
    1. learning_rate : 일반적으로 0.01 ~ 0.1로 두고 다른 파라미터를 튜닝함. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
    2. num_iterations : 트리 개수
    3. max_depth : -1이면 제한없이 분기함. feature가 많다면 크게 설정
    4. boosting : 부스팅 방법
        - gbdt : (default) traditional gradient boosting decision tree
        - rf : random forest
        - dart : dropouts meet multiple additive regression trees
        - goss : gradient-based one-side sampling
    5. bagging_fraction
        - 배깅을 위해 데이터를 랜덤 샘플링하여 학습에 사용
        - (0,1] 사이의 값
    6. feature_fraction
        - 매 트리마다 다른 feature를 랜덤하게 추출하여 학습.
        - 과적합 방지와 학습 속도 개선에 영향
    7. scale_pos_weight
        - 클래스 불균형 데이터 셋에서 weight를 주는 방식
    8. early_stopping_round
    9. lambda_l1, lambda_l2 : 규제
    



In [14]:
import lightgbm as lgb

In [17]:
sub_train_ds = lgb.Dataset(X_sub_train_data_trans, label = y_sub_train_data)
valid_ds = lgb.Dataset(X_valid_data_trans, label = y_valid_data)
params = {'learning_rate':0.01,
          #'max_depth':16,
          'objective':'regression',
          'metric':'mse',
          'is_training_metric':True,
          #'num_leaves':144,
          'feature_fraction':0.7,
          'bagging_fraction':0.7,
          'bagging_freq':5,
          'seed':42}

In [25]:
model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.137903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 658020
[200]	valid_0's l2: 332681
[300]	valid_0's l2: 230092
[400]	valid_0's l2: 188065
[500]	valid_0's l2: 167045
[600]	valid_0's l2: 155207
[700]	valid_0's l2: 146020
[800]	valid_0's l2: 140124
[900]	valid_0's l2: 135435
[1000]	valid_0's l2: 131646
[1100]	valid_0's l2: 127993
[1200]	valid_0's l2: 125329
[1300]	valid_0's l2: 122595
[1400]	valid_0's l2: 120408
[1500]	valid_0's l2: 118525
[1600]	valid_0's l2: 116932
[1700]	valid_0's l2: 115454
[1800]	valid_0's l2: 114044
[1900]	valid_0's l2: 112964
[

In [26]:
# predict
train_pred = model.predict(X_sub_train_data_trans)
valid_pred = model.predict(X_valid_data_trans)

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0

In [27]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

훈련데이터 rmsle:  1.766076496281916
검증데이터 rmsle:  1.475224764978254


**베이지안 최적화**

In [18]:
from bayes_opt import BayesianOptimization

In [19]:
#!pip install bayesian-optimization

In [ ]:
sub_train_ds = lgb.Dataset(X_sub_train_data_trans, label = y_sub_train_data)
valid_ds = lgb.Dataset(X_valid_data_trans, label = y_valid_data)

In [26]:
def valid_msle(learning_rate, max_depth, feature_fraction, bagging_fraction, num_leaves):
    params = {'learning_rate':learning_rate,
          'max_depth': int(max_depth),
          'objective':'regression',
          'metric':'mse',
          'num_leaves': int(num_leaves),    
          'is_training_metric':True,
          'feature_fraction':feature_fraction,
          'bagging_fraction':bagging_fraction,
          'seed':42}
    model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)
    valid_pred = model.predict(X_valid_data_trans)
    valid_pred[valid_pred < 0] = 0
    return - np.sqrt(mean_squared_log_error(y_valid_data, valid_pred))

In [27]:
pbounds = {'learning_rate' :(0.01, 0.1),
           'max_depth': (20,40),
           'num_leaves' : (32,100),
           'feature_fraction':(0.5,0.9),
           'bagging_fraction':(0.5,0.9)}

gbm_bayes = BayesianOptimization(f = valid_msle, pbounds = pbounds, verbose = 2, random_state = 42)

gbm_bayes.maximize(init_points=5, n_iter = 50)
gbm_bayes.max

|   iter    |  target   | baggin... | featur... | learni... | max_depth | num_le... |
-------------------------------------------------------------------------------------


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.204497 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 131260
[200]	valid_0's l2: 115354
[300]	valid_0's l2: 108961
[400]	valid_0's l2: 105321
[500]	valid_0's l2: 103170
[600]	valid_0's l2: 101771
[700]	valid_0's l2: 100809
[800]	valid_0's l2: 100133
[900]	valid_0's l2: 99344.7
[1000]	valid_0's l2: 98891.4
[1100]	valid_0's l2: 98522.8
[1200]	valid_0's l2: 98298.8
Early stopping, best iteration is:
[1194]	valid_0's l2: 98285
|  1        | -1.384    |  0.6498   |  0.8803   |  0.07588  |  31.97    |  42.61    |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.203002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 134043
[200]	valid_0's l2: 120236
[300]	valid_0's l2: 113765
[400]	valid_0's l2: 110631
[500]	valid_0's l2: 109128
[600]	valid_0's l2: 107918
[700]	valid_0's l2: 107837
[800]	valid_0's l2: 107527
[900]	valid_0's l2: 107309
Early stopping, best iteration is:
[869]	valid_0's l2: 107127
|  2        | -1.413    |  0.5624   |  0.5232   |  0.08796  |  32.02    |  80.15    |


...

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.214563 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 316360
[200]	valid_0's l2: 165669
[300]	valid_0's l2: 135235
[400]	valid_0's l2: 124002
[500]	valid_0's l2: 118610
[600]	valid_0's l2: 114625
[700]	valid_0's l2: 112023
[800]	valid_0's l2: 110377
[900]	valid_0's l2: 108710
[1000]	valid_0's l2: 107568
[1100]	valid_0's l2: 106633
[1200]	valid_0's l2: 105992
[1300]	valid_0's l2: 105423
[1400]	valid_0's l2: 105026
[1500]	valid_0's l2: 104480
[1600]	valid_0's l2: 104117
[1700]	valid_0's l2: 103819
[1800]	valid_0's l2: 103325
[1900]	valid_0's l2: 103166
[

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.208629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 121944
[200]	valid_0's l2: 107527
[300]	valid_0's l2: 104102
[400]	valid_0's l2: 101643
[500]	valid_0's l2: 100468
[600]	valid_0's l2: 100235
[700]	valid_0's l2: 100027
[800]	valid_0's l2: 99748.1
[900]	valid_0's l2: 98910.1
[1000]	valid_0's l2: 98640
[1100]	valid_0's l2: 98603.8
Early stopping, best iteration is:
[1022]	valid_0's l2: 98535.9
|  55       | -1.244    |  0.5879   |  0.7996   |  0.05431  |  38.94    |  98.48    |


{'target': -1.2169151662447444,
 'params': {'bagging_fraction': 0.5765701214228706,
  'feature_fraction': 0.8613356302139816,
  'learning_rate': 0.034600572091497105,
  'max_depth': 39.13465006223704,
  'num_leaves': 98.47466625953058}}

In [28]:
gbm_bayes.res

[{'target': -1.3839376625214053,
  'params': {'bagging_fraction': 0.649816047538945,
   'feature_fraction': 0.8802857225639664,
   'learning_rate': 0.07587945476302646,
   'max_depth': 31.973169683940732,
   'num_leaves': 42.60926755008568}},
 {'target': -1.4128583485308996,
  'params': {'bagging_fraction': 0.562397808134481,
   'feature_fraction': 0.5232334448672797,
   'learning_rate': 0.08795585311974417,
   'max_depth': 32.022300234864176,
   'num_leaves': 80.14893529013109}},
 {'target': -1.320813494663103,
  'params': {'bagging_fraction': 0.508233797718321,
   'feature_fraction': 0.8879639408647977,
   'learning_rate': 0.08491983767203796,
   'max_depth': 24.246782213565524,
   'num_leaves': 44.36409777008284}},
 {'target': -1.3313554150739568,
  'params': {'bagging_fraction': 0.5733618039413735,
   'feature_fraction': 0.621696897183815,
   'learning_rate': 0.05722807884690141,
   'max_depth': 28.638900372842315,
   'num_leaves': 51.803581533466854}},
 {'target': -1.2968416748172

In [29]:
gbm_bayes.max

{'target': -1.2169151662447444,
 'params': {'bagging_fraction': 0.5765701214228706,
  'feature_fraction': 0.8613356302139816,
  'learning_rate': 0.034600572091497105,
  'max_depth': 39.13465006223704,
  'num_leaves': 98.47466625953058}}

In [30]:
params = {'learning_rate':0.03,
          'max_depth': 39,
          'objective':'regression',
          'metric':'mse',
          'num_leaves': 98,    
          'is_training_metric':True,
          'feature_fraction':0.8,
          'bagging_fraction':0.6,
          'seed':42}
model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.208161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 158034
[200]	valid_0's l2: 116452
[300]	valid_0's l2: 108218
[400]	valid_0's l2: 104525
[500]	valid_0's l2: 103162
[600]	valid_0's l2: 101762
[700]	valid_0's l2: 100586
[800]	valid_0's l2: 99872
[900]	valid_0's l2: 99217.1
[1000]	valid_0's l2: 98957.5
[1100]	valid_0's l2: 98574.8
[1200]	valid_0's l2: 98434.8
[1300]	valid_0's l2: 98193.5
[1400]	valid_0's l2: 97742.6
[1500]	valid_0's l2: 97543.9
[1600]	valid_0's l2: 97383
[1700]	valid_0's l2: 97296.1
[1800]	valid_0's l2: 97142
[1900]	valid_0's l2: 970

In [31]:
# predict
train_pred = model.predict(X_sub_train_data_trans)
valid_pred = model.predict(X_valid_data_trans)

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0

In [32]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

훈련데이터 rmsle:  1.3981113192176338
검증데이터 rmsle:  1.2331210297822568


**베이지안 최적화2**
- 위의 결과를 보았을 때 훈련데이터와 검증데이터의 rmsle의 값이 비슷하게 나타난다. 즉, 과소적합되었다고 예상해볼 수 있다. 따라서 규제를 조절하는 max_depth와 num_leaves를 더 큰 값을 주어 최적화를 시행한다.
- depth랑 num_leaves의 범위를 좀 더 크기가 큰 범위에서 시행


In [39]:
sub_train_ds = lgb.Dataset(X_sub_train_data_trans, label = y_sub_train_data)
valid_ds = lgb.Dataset(X_valid_data_trans, label = y_valid_data)

In [40]:
def valid_msle(learning_rate, max_depth, feature_fraction, bagging_fraction, num_leaves):
    params = {'learning_rate':learning_rate,
          'max_depth': int(max_depth),
          'objective':'regression',
          'metric':'mse',
          'num_leaves': int(num_leaves),    
          'is_training_metric':True,
          'feature_fraction':feature_fraction,
          'bagging_fraction':bagging_fraction,
          'seed':42}
    model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)
    valid_pred = model.predict(X_valid_data_trans)
    valid_pred[valid_pred < 0] = 0
    return - np.sqrt(mean_squared_log_error(y_valid_data, valid_pred))

In [41]:
pbounds = {'learning_rate' :(0.01, 0.1),
           'max_depth': (30,100),
           'num_leaves' : (80,10000),
           'feature_fraction':(0.5,0.9),
           'bagging_fraction':(0.4,0.8)}

gbm_bayes = BayesianOptimization(f = valid_msle, pbounds = pbounds, verbose = 2, random_state = 42)

gbm_bayes.maximize(init_points=5, n_iter = 100)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | num_le... |
-------------------------------------------------------------------------------------
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.201651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 101556
Early stopping, best iteration is:
[93]	valid_0's l2: 101273
|  1        | -1.066    |  0.5498   |  0.8803   |  0.07588  |  71.91    |  1.628e+0 |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.185290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	valid_0's l2: 144821
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's l2: 140142
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.198780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 102685
Early stopping, best iteration is:
[96]	valid_0's l2: 102477
|  3        | -1.033    |  0.4082   |  0.888    |  0.08492  |  44.86    |  1.884e+0 |


...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.193474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 141205
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's l2: 128320
[300]	valid_0's l2: 125492
[400]	valid_0's l2: 125049
[500]	valid_0's l2: 125258
Early stopping, best iteration is:
[410]	valid_0's l2: 124900
|  103      | -1.296    |  0.6437   |  0.5386   |  0.05333  |  90.69    |  1.482e+0 |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.207056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 102099
Early stopping, best iteration is:
[65]	valid_0's l2: 101162
|  104      | -1.032    |  0.7411   |  0.8775   |  0.08436  |  86.15    |  2.45e+03 |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.192276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 133580
[200]	valid_0's l2: 128506
Early stopping, best iteration is:
[196]	valid_0's l2: 128404
|  105      | -1.316    |  0.4367   |  0.6268   |  0.05953  |  65.53    |  9.596e+0 |


In [42]:
gbm_bayes.res

[{'target': -1.0662770582676169,
  'params': {'bagging_fraction': 0.5498160475389451,
   'feature_fraction': 0.8802857225639664,
   'learning_rate': 0.07587945476302646,
   'max_depth': 71.90609389379256,
   'num_leaves': 1627.7049131889703}},
 {'target': -1.4229612246240815,
  'params': {'bagging_fraction': 0.46239780813448106,
   'feature_fraction': 0.5232334448672797,
   'learning_rate': 0.08795585311974417,
   'max_depth': 72.07805082202461,
   'num_leaves': 7104.079971736771}},
 {'target': -1.0329331976561584,
  'params': {'bagging_fraction': 0.408233797718321,
   'feature_fraction': 0.8879639408647977,
   'learning_rate': 0.08491983767203796,
   'max_depth': 44.863737747479334,
   'num_leaves': 1883.7036746944382}},
 {'target': -1.2763142258961804,
  'params': {'bagging_fraction': 0.47336180394137356,
   'feature_fraction': 0.621696897183815,
   'learning_rate': 0.05722807884690141,
   'max_depth': 60.2361513049481,
   'num_leaves': 2968.993070764576}},
 {'target': -1.28258875854

In [43]:
gbm_bayes.max

{'target': -0.9371455938031996,
 'params': {'bagging_fraction': 0.5249451271879992,
  'feature_fraction': 0.8783531742224099,
  'learning_rate': 0.02981736012953757,
  'max_depth': 43.15189542920939,
  'num_leaves': 8388.611279100138}}

In [44]:
params = {'learning_rate':0.03,
          'max_depth': 43,
          'objective':'regression',
          'metric':'mse',
          'num_leaves': 8388,    
          'is_training_metric':True,
          'feature_fraction':0.9,
          'bagging_fraction':0.5,
          'seed':42}
model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.206367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1173
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 104
[LightGBM] [Info] Start training from score 326.506259
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 111775
[200]	valid_0's l2: 98854.7
Early stopping, best iteration is:
[189]	valid_0's l2: 98759.2


In [45]:
# predict
train_pred = model.predict(X_sub_train_data_trans)
valid_pred = model.predict(X_valid_data_trans)

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0

In [46]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

훈련데이터 rmsle:  0.8976957106022574
검증데이터 rmsle:  0.9571586389785375


- 음...

**full_train, test**

In [80]:
test_data_trans.head()

,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekday,trend,NewYearsDay,Earthquake,...,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,type_y_B,type_y_C,type_y_D,type_y_E
0,0.000000,0.244108,0.172763,13,2017,8,2,1688,False,False,...,0,0,0,0,0,0,0,0,1,0
1,0.000000,0.244108,0.172763,13,2017,8,2,1688,False,False,...,0,0,0,0,0,0,0,0,1,0
2,0.002699,0.244108,0.172763,13,2017,8,2,1688,False,False,...,0,0,0,0,0,0,0,0,1,0
3,0.026991,0.244108,0.172763,13,2017,8,2,1688,False,False,...,0,0,0,0,0,0,0,0,1,0
4,0.000000,0.244108,0.172763,13,2017,8,2,1688,False,False,...,0,0,0,0,0,0,0,0,1,0


In [48]:
train_ds = lgb.Dataset(X_train_data_trans, label = y_train_data)

In [49]:
params = {'learning_rate':0.03,
          'max_depth': 43,
          'objective':'regression',
          'metric':'mse',
          'num_leaves': 8388,    
          'is_training_metric':True,
          'feature_fraction':0.9,
          'bagging_fraction':0.5,
          'seed':42}

model = lgb.train(params, train_ds, 200, verbose_eval = 100)

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.291189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1206
[LightGBM] [Info] Number of data points in the train set: 3054348, number of used features: 104
[LightGBM] [Info] Start training from score 359.020892


In [82]:
# predict
train_pred = model.predict(X_train_data_trans)
test_pred = model.predict(test_data_trans)

train_pred[train_pred < 0] = 0
test_pred[test_pred < 0] = 0

In [83]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_train_data, train_pred)))

훈련데이터 rmsle:  0.8811268669849754


In [84]:
submission = pd.read_csv('sample_submission.csv')

In [85]:
submission['sales'] = test_pred

In [86]:
submission.to_csv('submission_gbm1.csv', index = False) # 0.71810

**hybrid model (['Year', 'Month','Weekday', 'trend'])**

**1.linear regression**

In [44]:
X_sub_train_data.head()

,store_nbr,family,onpromotion,dcoilwtico,transactions,type_y,cluster,Year,Month,Weekday,trend,NewYearsDay,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre,no_work_day
0,1,AUTOMOTIVE,0,93.14,1420.876311,D,13,2013,1,1,0,True,False,False,False,False,False,True
1,1,BABY CARE,0,93.14,1420.876311,D,13,2013,1,1,0,True,False,False,False,False,False,True
2,1,BEAUTY,0,93.14,1420.876311,D,13,2013,1,1,0,True,False,False,False,False,False,True
3,1,BEVERAGES,0,93.14,1420.876311,D,13,2013,1,1,0,True,False,False,False,False,False,True
4,1,BOOKS,0,93.14,1420.876311,D,13,2013,1,1,0,True,False,False,False,False,False,True


In [51]:
X_sub_train_data_trans_lin = X_sub_train_data_trans[['Year', 'Month','Weekday', 'trend']]
X_valid_data_trans_lin = X_valid_data_trans[['Year', 'Month','Weekday', 'trend']]
X_train_data_trans_lin = X_train_data_trans[['Year', 'Month','Weekday', 'trend']]
test_data_trans_lin = test_data_trans[['Year', 'Month','Weekday', 'trend']]

In [46]:
# modeling
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_sub_train_data_trans_lin, y_sub_train_data)

LinearRegression()

In [47]:
# predict
train_pred_lin = model.predict(X_sub_train_data_trans_lin)
valid_pred_lin = model.predict(X_valid_data_trans_lin)

In [67]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred_lin)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred_lin)))

훈련데이터 rmsle:  4.005683206316629
검증데이터 rmsle:  3.7405319749075763


In [49]:
y_sub_train_after_lin = y_sub_train_data - train_pred_lin
y_valid_after_lin = y_valid_data - valid_pred_lin

**2. lightGBM**

In [50]:
X_sub_train_data_trans_gbm = X_sub_train_data_trans.drop(columns = ['Year', 'Month','Weekday', 'trend'])
X_valid_data_trans_gbm = X_valid_data_trans.drop(columns = ['Year', 'Month','Weekday', 'trend'])
X_train_data_trans_gbm = X_train_data_trans.drop(columns = ['Year', 'Month','Weekday', 'trend'])
test_data_trans_gbm = test_data_trans.drop(columns = ['Year', 'Month','Weekday', 'trend'])

In [55]:
import re
X_sub_train_data_trans_gbm = X_sub_train_data_trans_gbm.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_valid_data_trans_gbm = X_valid_data_trans_gbm.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train_data_trans_gbm = X_train_data_trans_gbm.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_data_trans_gbm = test_data_trans_gbm.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [56]:
sub_train_ds = lgb.Dataset(X_sub_train_data_trans_gbm, label = y_sub_train_after_lin)
valid_ds = lgb.Dataset(X_valid_data_trans_gbm, label = y_valid_after_lin)

In [57]:
params = {'learning_rate':0.03,
          'max_depth': 43,
          'objective':'regression',
          'metric':'mse',
          'num_leaves': 8388,    
          'is_training_metric':True,
          'feature_fraction':0.9,
          'bagging_fraction':0.5,
          'seed':42}
model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)

C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.183501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	valid_0's l2: 167552
[200]	valid_0's l2: 148648
Early stopping, best iteration is:
[185]	valid_0's l2: 148571


In [69]:
# predict
train_pred_gbm = model.predict(X_sub_train_data_trans_gbm)
valid_pred_gbm = model.predict(X_valid_data_trans_gbm)

**final(linear regression + lightgbm)**

In [70]:
train_pred = train_pred_lin + train_pred_gbm
valid_pred = valid_pred_lin + valid_pred_gbm

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0

In [71]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_sub_train_data, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

훈련데이터 rmsle:  1.7348018353007544
검증데이터 rmsle:  2.661123136486128


**최적화**

In [66]:
def valid_msle(learning_rate, max_depth, feature_fraction, bagging_fraction, num_leaves):
    params = {'learning_rate':learning_rate,
          'max_depth': int(max_depth),
          'objective':'regression',
          'metric':'mse',
          'num_leaves': int(num_leaves),    
          'is_training_metric':True,
          'feature_fraction':feature_fraction,
          'bagging_fraction':bagging_fraction,
          'seed':42}
    model = lgb.train(params, sub_train_ds, 3000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)
    valid_pred_gbm = model.predict(X_valid_data_trans_gbm)
    valid_pred = valid_pred_lin + valid_pred_gbm
    valid_pred[valid_pred < 0] = 0
    return - np.sqrt(mean_squared_log_error(y_valid_data, valid_pred))

In [74]:
pbounds = {'learning_rate' :(0.01, 0.1),
           'max_depth': (20,100),
           'num_leaves' : (50,1000),
           'feature_fraction':(0.5,0.9),
           'bagging_fraction':(0.4,0.8)}

gbm_bayes = BayesianOptimization(f = valid_msle, pbounds = pbounds, verbose = 2, random_state = 42)

gbm_bayes.maximize(init_points=5, n_iter = 20)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | num_le... |
-------------------------------------------------------------------------------------


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.188562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 143025
[200]	valid_0's l2: 142754
Early stopping, best iteration is:
[149]	valid_0's l2: 140240
|  1        | -2.665    |  0.5498   |  0.8803   |  0.07588  |  67.89    |  198.2    |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.411793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 181049
[200]	valid_0's l2: 175229
[300]	valid_0's l2: 176288
Early stopping, best iteration is:
[212]	valid_0's l2: 174710
|  2        | -2.669    |  0.4624   |  0.5232   |  0.08796  |  68.09    |  722.7    |


...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.165881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 150809
Early stopping, best iteration is:
[85]	valid_0's l2: 149365
|  23       | -2.68     |  0.6144   |  0.8207   |  0.08434  |  63.95    |  568.9    |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.206358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 333301
[200]	valid_0's l2: 206965
[300]	valid_0's l2: 181454
[400]	valid_0's l2: 174334
[500]	valid_0's l2: 171698
[600]	valid_0's l2: 170737
[700]	valid_0's l2: 170561
[800]	valid_0's l2: 169922
[900]	valid_0's l2: 169309
[1000]	valid_0's l2: 169268
Early stopping, best iteration is:
[941]	valid_0's l2: 169075
|  24       | -2.67     |  0.4275   |  0.5277   |  0.02379  |  47.06    |  404.5    |


C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\82104\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.186996 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 893
[LightGBM] [Info] Number of data points in the train set: 2368278, number of used features: 100
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 177767
[200]	valid_0's l2: 158270
[300]	valid_0's l2: 157965
Early stopping, best iteration is:
[257]	valid_0's l2: 157355
|  25       | -2.672    |  0.5746   |  0.5604   |  0.06204  |  77.99    |  179.1    |


In [75]:
gbm_bayes.res

[{'target': -2.6649309980993356,
  'params': {'bagging_fraction': 0.5498160475389451,
   'feature_fraction': 0.8802857225639664,
   'learning_rate': 0.07587945476302646,
   'max_depth': 67.89267873576293,
   'num_leaves': 198.2177084203147}},
 {'target': -2.668516376174093,
  'params': {'bagging_fraction': 0.46239780813448106,
   'feature_fraction': 0.5232334448672797,
   'learning_rate': 0.08795585311974417,
   'max_depth': 68.0892009394567,
   'num_leaves': 722.6689489062433}},
 {'target': -2.669147844462503,
  'params': {'bagging_fraction': 0.408233797718321,
   'feature_fraction': 0.8879639408647977,
   'learning_rate': 0.08491983767203796,
   'max_depth': 36.987128854262096,
   'num_leaves': 222.73371884674557}},
 {'target': -2.690339068590749,
  'params': {'bagging_fraction': 0.47336180394137356,
   'feature_fraction': 0.621696897183815,
   'learning_rate': 0.05722807884690141,
   'max_depth': 54.55560149136926,
   'num_leaves': 326.6676831881398}},
 {'target': -2.667226755939819

In [76]:
gbm_bayes.max

{'target': -2.661525137045119,
 'params': {'bagging_fraction': 0.6929162269702784,
  'feature_fraction': 0.8107785553570085,
  'learning_rate': 0.08852165335753068,
  'max_depth': 37.25118101806793,
  'num_leaves': 129.61395114030773}}

**베이지안 최적화3**
- 안 건드린 다른 파라미터를 더 건들여보기